# pyro

 ---

**example1**：

某种元件的寿命$X$（以$h$计）服从正态分布$N(\mu, \sigma^2)$，$\mu, \sigma^2$均未知。现在测得$16$只元件得寿命如下：

|0|1|2|3|4|5|6|7|8|9|A|B|C|D|E|F|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|222|362|168|250|149|260|485|170|159|280|101|212|224|379|179|264|

请问是否有理由认为元件的平均寿命大于$255h$?

The life $X$ of a component (measured by hours) obeys a normal distribution $N(\mu, \sigma^2)$, $\mu, \sigma^2$ are both unknown. Now the life of 16 components is as follows:

|0|1|2|3|4|5|6|7|8|9|A|B|C|D|E|F|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|222|362|168|250|149|260|485|170|159|280|101|212|224|379|179|264|

Is there any reason to think that the average life of components is greater than 255 hours?

In [5]:
import torch
import pyro
import numpy as np
import pyro.distributions as dist
import matplotlib.pyplot as plt
import scipy.stats as st

In [12]:
X = np.array([159,280,101,212,224,379,179,264, 222,362,168,250,149,260,485,170], dtype=np.double)


# 在正态总体𝜎2未知的情况下检验𝜇
def t_test(X, confidence, mu_0,sigma_2=None):
    '''
    在正态总体𝜎2未知的情况下检验𝜇
    H0:𝜇≤𝜇0  H1:𝜇>𝜇0
    
    𝑡=(𝑥̂ −𝜇0)/(𝑠/√n) ≥ 𝑡𝛼(n-1)
    '''
    N = len(X)
    
    hat_x = X.mean()
    
    _X = X - hat_x
    
    s = np.sqrt(np.dot(_X, _X.T)/(N-1))
    
    t = (hat_x - mu_0)/(s/np.sqrt(N))
    t0 = st.t.ppf(1-confidence, N-1)
    
    return t < t0
    

if t_test(X,  0.05, 225):
    print('t没有落在拒绝域中，故接受H0:𝜇≤𝜇0=225')
else:
    print('t落在拒绝域中，故接受H0:𝜇≤𝜇0=225')

t没有落在拒绝域中，故接受H0:𝜇≤𝜇0=225


**example2**:

做以下实验比较人对红光或绿光的反应时间（以s记）.实验在点亮红光或绿光的同时，启动计时器，要求受测试者见到红光或绿光点亮时，就按下按钮，切断计时器，这就能测得反应时间.测量的结果如下表：

|红/绿光($x/y$)|1|2|3|4|5|6|7|8|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|红光($x$)|0.30|0.23|0.41|0.53|0.24|0.36|0.38|0.51|
|绿光($y$)|0.43|0.32|0.58|0.46|0.27|0.41|0.38|0.61|
|$d$ $=$ $x$ $-$ $y$|-0.13|-0.09|-0.17|0.07|-0.03|-0.05|0.00|-0.10|

设$D_i=X_i-Y_i, (i=1,2,\cdots, 8)$是来自正态总体$N(\mu_D, \sigma^2_D)$的样本，$\mu_D, \sigma^2_D$均未知.试检验假设（取显著水平$\alpha=0.05$）

$$
H_0:\mu_D \geq 0, H_1:\mu_D < 0.
$$

The following experiments were done to compare the reaction time of people to red or green light (recorded in s). The experiment started a timer while lighting red or green light. When the subjects saw red or green light lighting, they were asked to press a button and cut off the timer. This can measure the reaction time. The results of the measurement are as follows:

|Red/Green($x/y$)|1|2|3|4|5|6|7|8|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|Red($x$)|0.30|0.23|0.41|0.53|0.24|0.36|0.38|0.51|
|Green($y$)|0.43|0.32|0.58|0.46|0.27|0.41|0.38|0.61|
|$d$ $=$ $x$ $-$ $y$|-0.13|-0.09|-0.17|0.07|-0.03|-0.05|0.00|-0.10|

Let $D_i=X_i-Y_i, (i=1,2,\cdots, 8)$ be a sample from a normal population$N(\mu_D, \sigma^2_D)$, $\mu_D, \sigma^2_D$ both of which are unknown. Test the hypothesis (take the significant level = 0.05).

$$
H_0:\mu_D \geq 0, H_1:\mu_D < 0.
$$

In [2]:
import torch
import pyro
import numpy as np
import pyro.distributions as dist
import matplotlib.pyplot as plt
import scipy.stats as st

In [20]:
n = 8
alpha = 0.05
X = [0.30, 0.23, 0.41, 0.53, 0.24, 0.36, 0.38, 0.51]
Y = [0.43, 0.32, 0.58, 0.46, 0.27, 0.41, 0.38, 0.61]
d = [-0.13, -0.09, -0.17, 0.07, -0.03, -0.05, 0.00, -0.10]

X = np.array(X)
Y = np.array(Y)
D = X-Y

x_d = D.mean()
s_d = D.std()

t = x_d / (s_d / np.sqrt(n))
t_alpha = st.t.ppf(1- alpha, n-1)

if t < -1 * t_alpha:
    print("拒绝H0, 𝜇𝐷<0, 即人对红光的反应时间小于绿光的反应时间，也就是人对红光的反应时间要比绿光快。")

拒绝H0, 𝜇𝐷<0, 即人对红光的反应时间小于绿光的反应时间，也就是人对红光的反应时间要比绿光快。


**example3:**

某厂生产的某种型号的电池，其寿命（以$h$ 计）长期以来服从方差($\sigma^2=5000$ 的正态分布，现有一批这种电池，从它的生产情况来看，寿命的波动性有所改变． 现随机取$26$只电池，测出其寿命的样本方差$s^2=9200$. 问根据这一数据能否推断这批电池的寿命的波动性较以往的有显著的变化（取$\alpha=0.02$)?

The life of a certain type of batteries produced by a factory has long obeyed the normal distribution of variance ($\sigma^2=5000$). The fluctuation of life of some batteries has changed according to their production situation. $26$ batteries are randomly selected and the sample variance of life is $s^2=9200$. Can we infer the fluctuation of life of these batteries from this data? Has there been a significant change in the past ($\alpha = 0.02$)?

In [1]:
import torch
import pyro
import numpy as np
import pyro.distributions as dist
import matplotlib.pyplot as plt
import scipy.stats as st

In [11]:
# H0:sigma^2 = 5000 H1: sigma^2 != 5000
alpha=0.02
n = 26
sigma_0 = 5000
s_2 = 9200


# (n-1)s^2/(sigma^2)  ~  𝜒^2(n-1)

reject_field_right=st.chi2.ppf(1-alpha/2, n-1)
reject_field_left=st.chi2.ppf(alpha/2, n-1)

watch_value = (n-1)*s_2/sigma_0

if reject_field_left <= watch_value <=reject_field_right:
    print("接受H0,认为这批电池寿命的波动性较以往的没有显著的变化．")
else:
    print("拒绝H0, 认为这批电池寿命的波动性较以往的有显著的变化．")




拒绝H0, 认为这批电池寿命的波动性较以往的有显著的变化．


---

**example2**: 大数定定律 

**伯努利大数定律**：

设$m$和$n$重伯努利实验中事件$A$出现的次数，$p$是$A$在每次实验中发生的概率，则对任意给定的实数$\epsilon>0$,则有
$$
\underset{n\rightarrow\infty}{lim}P\left\{\left|\frac{m}{n}-p\right|<\epsilon\right\}=1
$$

In [13]:
import torch
import pyro
import numpy as np
import pyro.distributions as dist
import matplotlib.pyplot as plt
import scipy.stats as st

In [88]:
N = 100000000
p = 0.1
epsilon = 1e-4

d = dist.Binomial(N, p)
m = d.sample()

hat_X = m/N

error = torch.abs(m/N - p)

flag = error.item() < epsilon

if flag is True:
    print("hat_X:", hat_X.item(),  "E(X):", p, "Error:", error.item())

hat_X: 0.10000672936439514 E(X): 0.1 Error: 6.727874279022217e-06
